In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import csv
import os
import sys
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [5]:
# list working dir

print(os.getcwd())

C:\Users\micha\OneDrive - České vysoké učení technické v Praze\Dokumenty\PycharmProjects\optimizin\General-Optimization-Solver\src\vrp\old


In [9]:
# renaming results


def process_json_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                data = json.load(file)
                
            run_history = data.pop('run_history')
            
            for run in run_history:
                if 'solver_type' in run.keys() and run['solver_type'] == 'GA':
                    run['solution_progress'] = [run['solution_value']*10, run['solve_time'], -1]

            # Seřadíme záznamy ve slovníku
            data = {
                'instance_kind': 'VRPTW',
                'benchmark_name': 'solomon_25',
                'instance_name': data.pop('instance_name'),  # Consistency with original source of Solomon instances
                'reference_solution': data.pop('reference_solution'),
                'data': data.pop('data'),
                'run_history': run_history,
            }
            
            # Uložíme změny zpět do JSON souboru
            with open(file_path, 'w') as file:
                json.dump(data, file, indent=4)
        # break

folder_path = "..\\..\\data\\VRPTW\\solomon_25\\"
process_json_files(folder_path)


In [8]:
# merge results

folder = "..\\..\\output\\"

subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
print(subfolders)

# use the first subfolder as target and copy the last results to them

for benchmark in os.scandir(subfolders[-1]):
    if benchmark.is_dir():
        print(benchmark.path)
        for file in os.scandir(benchmark):
            file_path = file.path
            if file_path.endswith('.json'):
                print(file_path)
                with open(file, 'r') as f:
                    target = json.load(f)
                for folder in subfolders[:-1]:
                    if not os.path.exists(os.path.join(folder, benchmark.name, file.name)):
                        continue
                    with open(os.path.join(folder, benchmark.name, file.name), 'r') as f:
                        source = json.load(f)
                    new = source['run_history'][-1]
                    if new['solver_name'] == 'GA PSO, decode_chromosome_fast2':
                        new['solver_name'] = 'GA PSO, decode_chromosome_fast'
                    target['run_history'].append(new)
                with open(file_path, 'w') as f:
                    json.dump(target, f, indent=4)
                    

['..\\..\\output\\1', '..\\..\\output\\2', '..\\..\\output\\my_output']
..\..\output\my_output\solomon_100
..\..\output\my_output\solomon_100\C101.json
..\..\output\my_output\solomon_100\C102.json
..\..\output\my_output\solomon_100\C103.json
..\..\output\my_output\solomon_100\C104.json
..\..\output\my_output\solomon_100\C105.json
..\..\output\my_output\solomon_100\C106.json
..\..\output\my_output\solomon_100\C107.json
..\..\output\my_output\solomon_100\C108.json
..\..\output\my_output\solomon_100\C109.json
..\..\output\my_output\solomon_100\C201.json
..\..\output\my_output\solomon_100\C202.json
..\..\output\my_output\solomon_100\C203.json
..\..\output\my_output\solomon_100\C204.json
..\..\output\my_output\solomon_100\C205.json
..\..\output\my_output\solomon_100\C206.json
..\..\output\my_output\solomon_100\C207.json
..\..\output\my_output\solomon_100\C208.json
..\..\output\my_output\solomon_100\R101.json
..\..\output\my_output\solomon_100\R102.json
..\..\output\my_output\solomon_100\R10